## 模型构建的一般流程

### 前置包的导入

In [35]:
import tensorflow as tf
from tensorflow import keras
# 引用经典数据集时的 datasets，适用容器 Sequential, 网络层的父类 layers，网络的父类 Model,优化器 optimizers
# 损失类 loss
from tensorflow.keras import datasets, Sequential, layers,optimizers, metrics

### 数据预处理

In [36]:
#以图片数据为例，适用于全连接层
def preprocess(x, y):
    """
    自定义预处理函数
    参数：
    x -- 待处理的数据集,维度[b,28,28]
    y -- 待处理的数据集,[b]

    返回值：
    x -- 标准化和扁平化后的 x
    y -- 转换为 one_hot 向量
    """
    x = tf.cast(x, dtype=tf.float32) / 255. #标准化0-1
    #扁平化-不建议，后续需要用 squeeze函数处理。者此处不设，而在输入的时候对x设置x.reshape(x,(-1, 28*28))
    #  注意与 [-1, 28*28]的区别
    x = tf.reshape(x, [28*28])

    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)

    return x,y

### 数据集加载

拓展：可以考虑其他测试集进行，如 fashion_mnist,相比于 mnist 复杂

In [37]:
(x,y), (x_test, y_test)=datasets.mnist.load_data()
print('数据集mnist中x的维度', x.shape, '数据集mnist中y的维度', y.shape)
#数据加载后，需要转换为 Dataset对象
train_db = tf.data.Dataset.from_tensor_slices((x, y))
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# db = db.step1().step2().step3()
# shuffle(缓冲区大小，一般是一个较大的常数)，btach(一批的样本数量)
train_db = train_db.map(preprocess).shuffle(10000).batch(128)
test_db = test_db.map(preprocess).batch(128)

数据集mnist中x的维度 (60000, 28, 28) 数据集mnist中y的维度 (60000,)


In [38]:
# 数据维度查看
db_iter = iter(train_db)
sample = next(db_iter)
print('批样本 x 维度:', sample[0].shape,"批样本 y 维度", sample[1].shape)

批样本 x 维度: (128, 784) 批样本 y 维度 (128, 10)


### 模型的构建

拓展：增加网络的层数，观察损失和准确率

In [33]:
model = tf.keras.Sequential([ 
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)])
model.build(input_shape=[None, 28*28])
# 将网络结构显示出来
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


### 梯度下降和优化

均方误差函数

$$MSE(y, out)=\frac{1}{m} \sum_{i=1}^m (y_i - out_i)^2 $$

In [ ]:
# 高层 API 实现
model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# 训练并测试
# validation_freq=2 表示每循环两次epoch进行测试
model.fit(train_db, epochs=10, validation_data=test_db, validation_freq=2)
# 使用已经训练好的模型来评估测试数据集的性能
model.evaluate(test_db)